[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/change_permissions.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/change_permissions.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fchange_permissions.ipynb
[Run in Google Colab]: https://colab.research.google.com/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/change_permissions.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png)][Run in Google Colab] |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] | [Run in Google Colab] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It allows you to change the permissions of one particular user in one particular company, by selecting from the list of available permissions.

### Requirements
* If you are running in Jupyter Notebook, you don't need to make any code changes.
* If you are running in Jupyter Lab, you will need to install this JupyterLab extension:
  * @jupyter-widgets/jupyterlab-manager
* You need to have an existing user account on the WA/DMP instance.

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells *one at a time* (Shift-Enter).
(For this particular notebook, you *can't* use the 'Run All' command!)

Some cells prompt for user input.
When the "Select required permissions" cell is run, you can select multiple permissions from the list by Ctrl-click'ing.


## Setup
This may take a minute ...

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

import requests
import json
import ipywidgets as widgets
from datetime import datetime
import sys

## Global variables

In [ ]:
BASE_URL = 'https://gateway.wadmp.com'
BASE_PATH = 'api'
SESSION = requests.Session()  # Use one HTTPS session for all API calls

PERMISSION_NAMES = [
    "ViewUsers",
    "CreateCompany",
    "ManageRoles",
    "DeleteUser",
    "CreateUser",
    "DeleteCompany",
    "EditCompany",
    "ViewCompany",
    "EditUser",
    "ViewDevice",
    "CreateDevice",
    "ClaimDevice",
    "ViewDMServer",
    "AdmDMServer",
    "ChangeDeviceDMServer",
    "EditDevice",
    "ManageAppStore",
    "ManageAPIClients",
    "ManageSyncEngine",
    "DeleteDevice"
]

## Functions to be used later

In [ ]:
def login(username, password):
    """Login to the system, and return a token
    """
    url = f"{BASE_URL}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"Sending POST request to {url} with:\n"
          f"    credentials={credentials}\n")
    response = SESSION.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def get_companies(name=None):
    """Gets the list of companies in the system.
    """
    url = f"{BASE_URL}/{BASE_PATH}/companies"
    query = {'name': name}
    print(f"Sending GET request to {url} with:\n"
          f"    name={name}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of companies!")
        return None

    
def get_users(email=None, companies=[]):
    """Gets the list of users in a company or companies.
    """
    url = f"{BASE_URL}/{BASE_PATH}/users"
    query = {'email': email, 'companies': companies}
    print(f"Sending GET request to {url} with:\n"
          f"    email={email}\n"
          f"    companies={companies}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of users!")
        return None


def update_user_companies(user_id, companies):
    """Updates existing user's companies
    """
    url = f"{BASE_URL}/{BASE_PATH}/users/{user_id}/companies"
    model = {
      "id": user_id,
      "companies": companies
    }
    print(f"\nSending PUT request to {url} with:\n"
          f"    model={model}\n")
    response = SESSION.put(url, json=model)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return True
    else:
        print("Failed to create update user's companies!")
        return False

## User input

In [ ]:
USER_EMAIL = input("Enter the email address of the user whose permissions you want to change:")

COMPANY_NAME = input("Enter the name of the company to which this user belongs:")

### Select required permissions

In [ ]:
selected_permissions = widgets.SelectMultiple(
    options=PERMISSION_NAMES,
    rows=20,
    disabled=False
)
display(selected_permissions)

Ctrl-click to select from the list displayed, then run the next cell when you are ready ...

In [ ]:
new_permissions = 0
for selection in selected_permissions.value:
    weight = 2**(PERMISSION_NAMES.index(selection))
    print(f"- {selection} has weight {weight}")
    new_permissions += weight

binary_string = format(int(new_permissions), '020b')
print(f"Total = {new_permissions} decimal (= {binary_string} binary)")

## Login to server

In [ ]:
USERNAME = input("Enter YOUR WebAccess/DMP username:")
PASSWORD = input("Enter password:")
user_token = login(USERNAME, PASSWORD)
SESSION.headers.update({'Authorization': f'Bearer {user_token}'})

## Get Company ID

In [ ]:
matching_companies = get_companies(COMPANY_NAME)
if len(matching_companies) == 0:
    print("Company name not found!")
    sys.exit(1)
elif len(matching_companies) == 1:
    company_id = matching_companies[0]['id']
    print(f"{COMPANY_NAME} has company ID {company_id}")
else:
    print("More than one company matches that name! Just in case, we'll stop here.")
    sys.exit(1)

## Find User in Company

In [ ]:
matching_users = get_users(USER_EMAIL, [company_id])
if len(matching_users) == 0:
    print("User email not found!")
    sys.exit(1)
elif len(matching_users) == 1:
    user_id = matching_users[0]['id']
    print(f"{USER_EMAIL} has user ID {user_id}")
else:
    print("More than one user matches that email address! Just in case, we'll stop here.")
    sys.exit(1)

## Confirm change in permissions

In [ ]:
old_permissions = matching_users[0]['companies'][0]['permissions']
if "None" in old_permissions:
    old_permissions.remove('None')

old_permissions_widget = widgets.SelectMultiple(
    description='OLD:',
    options=PERMISSION_NAMES,
    value=old_permissions,
    rows=20,
    disabled=True
)

new_permissions_widget = widgets.SelectMultiple(
    description='NEW:',
    options=PERMISSION_NAMES,
    value=selected_permissions.value,
    rows=20,
    disabled=True
)

widgets.HBox([old_permissions_widget, new_permissions_widget])

If you are ready to make this change, run the next cell ...

In [ ]:
# For the update endpoint, we need to supply the full array of companies for the user,
# not just the one we want to change.

old_companies_array = get_users(USER_EMAIL)[0]['companies']
new_companies_array = old_companies_array
for index, company in enumerate(old_companies_array):
    if company['company']['name'] == COMPANY_NAME:
        company['permissions'] = selected_permissions.value
        new_companies_array[index] = company
        break

update_user_companies(user_id, new_companies_array)